In [ ]:
!mkdir -p ~/agave/funwave-tvd-jenkins-pipeline

%cd ~/agave/funwave-tvd-jenkins-pipeline

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

!auth-tokens-refresh


# Commit Our App Configuration
## Everything as Code
From here on out, every change we make will be committed to the Funwave repository, so that we can adhere to the _Everything as Code_ principle. Everything as Code is a powerful practice, as it allows for all aspects of the software lifecycle, like code, build process, documentation, etc., to be treated with the rigor as application code.

We will begin by adding a version tracker to our Funwave repository to better support build automation.

## Semantic Versioning
Our version tracker will use the very popular _Semantic Versioning_ schema. This schema composes a version out of 3 or 4 components, each with discrete meanings:
* **Major Version:** The first number in a semantic version increments when backwards-incompatible changes are made to the public API of a software package.
* **Minor Version:** The second number increments when backwards-compatible features are added to a package.
* **Patch Version:** The third number increments when bugfixes are made to software in a backwards-comptable manner.
* **Label:** The final _(optional)_ component of a semantic version is a label indicating build and release status.


Semantic versions offer a standardized way to communicate the evolution of a software package in a way that is meaningful to dependants. For more information on Semantic Versioning, visit the [official page](https://semver.org/).

The version of funwave we're using is currently pinned at `3.1.0`. Once we add the version tracker and start working on our build automation branch we're going to be working on version `3.2.0-dev`, as we're adding backwards-compatible features that are not yet ready for release.

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b dev"
writefile("version.txt","""3
2
0
dev""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F version.txt /home/jovyan/FUNWAVE-TVD/

In [ ]:
!ssh sandbox "git config --global user.email ${AGAVE_USERNAME}@sc18-demo.org"
!ssh sandbox "git config --global user.name '${AGAVE_USERNAME}'"
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add version.txt && git commit -m 'Adding version tracker.' && git checkout -b build-automation"

# Improving Our App for Automation

## Incorporate Our Version Tracker
Now that we have a version tracker, let's restructure our Dockerfile, build wrapper, and build app to incorporate it. 

We'll start by modifying our Dockerfile to _copy_ the Funwave TVD code into the container, instead of cloning from the upstream repository as it was before. This ensures that the Dockerfile and resulting image won't drift from the version of the code they are intended to package. This also makes it possible to reproduce a given version of the container from the git repository, as the git history of the image and code reflect one another directly now.

In [ ]:
writefile("Dockerfile","""
FROM stevenrbrandt/science-base
MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>

ARG BUILD_DATE
ARG VERSION

LABEL org.agaveplatform.ax.architecture="x86_64"                                \
      org.agaveplatform.ax.build-date="\$BUILD_DATE"                             \
      org.agaveplatform.ax.version="\$VERSION"                             \
      org.agaveplatform.ax.name="${AGAVE_USERNAME}/funwave-tvd"    \
      org.agaveplatform.ax.summary="Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi." \
      org.agaveplatform.ax.vcs-type="git"                                       \
      org.agaveplatform.ax.vcs-url="https://github.com/fengyanshi/FUNWAVE-TVD" \
      org.agaveplatform.ax.license="BSD 3-clause"
      
USER root
RUN mkdir -p /home/install/FUNWAVE-TVD
RUN chown jovyan /home/install/FUNWAVE-TVD
COPY --chown jovyan ./* /home/install/FUNWAVE-TVD/
USER jovyan

WORKDIR /home/install/FUNWAVE-TVD/src
RUN perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

RUN mkdir -p /home/jovyan/rundir
WORKDIR /home/jovyan/rundir
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Dockerfile /home/jovyan/FUNWAVE-TVD/

## Applying the Version to Every Build
Let's also modify our build wrapper to construct the app version from the version tracker in a given commit, and then assign that constructed version to the Docker build arguments. After this change, we will have a single build wrapper that will always build a Docker image with the correct version for the commit.

In [ ]:
writefile("funwave-build-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

sudo docker build \
    --build-arg "BUILD_DATE=\${AGAVE_JOB_SUBMIT_TIME}" \
    --build-arg "VERSION=\${VERSION}" \
    --rm -t funwave-tvd:\${VERSION} .

docker inspect funwave-tvd:\${VERSION}

""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/FUNWAVE-TVD/

In [ ]:
writefile("funwave-build-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Builds the funwave docker image",
   "shortDescription":"Funwave docker build",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"automation/funwave-tvd-docker-automation",
   "templatePath":"funwave-build-wrapper.txt",
   "testPath":"test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-app.txt /home/jovyan/FUNWAVE-TVD/

# Build Automatically On Each Commit
We now have a build app that will build a Docker image containing Funwave at a given point in git history, and apply meaningful metadata about the app and version derived from repository contents. It's all repeatable too, so the next step is to perform this process automatically upon each repository events, like commits, merges, and so forth.

## Jenkins: An Automation Platform
Jenkins is a widely-used platform for task automation that evolved from a software build system. Jenkins is open-source, free-to-use, and its feature set is largely driven by third-party extensions. The platform forms the backbone of many CI/CD architectures.

There are many alternatives ranging from cloud services like CircleCI and Travis CI, to locally-installable software like GoCD. For this tutorial we'll be using Jenkins, as it it is locally-installable and tends to be the most widely used option.

## Components of a Build Pipeline
The fundamental component of Jenkins task automation is a job, which defines a series of steps that jenkins will execute when triggered. There are two primary types of jobs supported by Jenkins: the freestyle job, and the pipeline job. Freestyle jobs are the easiest to set up initially, but they are difficult to manage over time and they necessarily separate build and deployment code from application code, giving you a split perspective on your project.

Pipeline jobs provide a mechanism to package the steps you wish Jenkins to automate directly into your application repository, more closely following the Everything as Code principle. Pipeline jobs are the direction that the Jenkins community is moving towards, and makes up the foundation of Blue Ocean. We'll be focusing on pipeline jobs in this tutorial.

A Jenkins pipeline consists of the following components:
* **Jenkinsfile:** This is a Groovy script named `Jenkinsfile` that exists in the root level of your repository, and defines the steps that Jenkins will execute when a pipeline is started.
* **Source Repository:** The VCS system or repository that contains the content you wish to automate. In our case, it is the Funwave TVD repository containing the application code, build app, and Dockerfile.
* **Triggers:** These are events, often time-based events or git lifecycle events, that Jenkins will watch for. When a trigger event occurs, Jenkins will start the associated pipeline.
* **Build environment:** The environment that your build process will be executed in. In our case, it is the Agave build app.
* **Secrets and Config:** Credentials and site-specific configuration necessary for your build pipeline. There are a variety of strategies for managing these, some of which are discussed in the _Recommended Reading_ for this section. For simplicity, we will simply include config and secrets in the pipeline in the tutorial pipeline.
* **Notifications:** This is how Jenkins will _(or won't)_ alert users of build status. Often email, Slack, etc.

## Create Our Jenkins Pipeline
We have already configured a pipeline in the tutorial Jenkins instance for you, so all that's left is to upload the Jenkinsfile it will be looking for when triggered. You can view the Jenkinsfile [here](/edit/notebooks/Jenkinsfile): 

In [ ]:
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F /home/jovyan/notebooks/Jenkinsfile /home/jovyan/FUNWAVE-TVD/

### Triggering our Jenkins Pipeline From a Commit
Now that we've added a Jenkinsfile to our repository we need to set up a trigger that starts the Jenkins pipeline after a `git commit`. Normally, this trigger would either be a cron-style process that checks for changes to a repository at regular intervals, or a GitHub webhook our Jenkins server has been attached to. Since our local repository is neither publicly-accessible in its current nor do we have the time to wait for a polling interval, we will instead trigger the build pipeline post-commit by way of _Git Hooks_.

A Git Hook is a user-definable script that executes during particular git repository lifecycle events. Hooks are defined by adding an executable script bearing the named of a particular hook  to a directory named `$GIT_DIR/hooks`. Non-executable hook scripts are ignored. Further documentation on Git Hooks can be found on [git-scm](https://git-scm.com/docs/githooks).

For our use case, we'll want to use the `post-commit` hook, which will fire after each commit. The below hook script will perform a simple curl on the Jenkins URL for our pipeline to kick off the build. We'll also do a `git update-index` to make sure that the post-commit hook is always executable after cloning.

In [ ]:
# Post-commit hook to trigger Jenkins job via curl

import os

agave_username = os.environ.get('AGAVE_USERNAME', '')
agave_password = os.environ.get('AGAVE_PASSWORD', '')

script_template = """#!/bin/bash
wget --auth-no-challenge \
     --http-user='{}' \
     --http-password='{}' \
     'jenkins:8080/jenkins/job/funwave-build-pipeline/build?token=sc18-training-job'
"""

script_content = script_template.format(agave_username, agave_password)
with open('post-commit', 'w') as post_commit_script:
    post_commit_script.write(script_content)

!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/.git/hooks
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F post-commit /home/jovyan/FUNWAVE-TVD/.git/hooks/

In [ ]:
# Set the hook to executable so that it won't be ignored.
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD/ && chmod +x .git/hooks/post-commit"

# Commit Your Code, Start Your Pipeline
We've updated our build app for automation and added it to the repo, added a pipeline definition, and a mechanism to trigger the build on each commit. Now all that's left for us to do is commit our changes and watch the pipeline run! Once you've committed your changes, you can view the running pipeline by navigating to the Jenkins pipeline URL:

https://<username\>.sc18.training.agaveplatform.org/jenkins/job

In [ ]:
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add Dockerfile funwave-build-wrapper.txt funwave-build-app.txt Jenkinsfile"
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added jenkinsfile and post-commit hook.'"

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout dev && git merge --squash build-automation"